Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 8
Andres Quinto - 18288  
Marlon Hernández - 15177  

[Repositorio_aqui](https://github.com/AndresQuinto5/IA_LAB08.git)

### Tasks 1 - Teoría  

1. Investigar el **algoritmo AC-3** y su relación con el algoritmo de **backtracking search**  
    El **algoritmo AC-3** es un algoritmo de consistencia de arcos utilizado en problemas de satisfacción de restricciones (CSP). Su objetivo es reducir los dominios de las variables eliminando valores que no tienen soporte, es decir, que no pueden formar parte de una solución consistente con las restricciones. Por otro lado, **el algoritmo de backtracking search** es una técnica de búsqueda que intenta construir una solución paso a paso, retrocediendo cuando encuentra que una asignación de valores no lleva a una solución válida. La relación entre ambos es que **AC-3** puede utilizarse antes del **backtracking** para preprocesar el CSP, reduciendo los dominios y, por lo tanto, el número de asignaciones a probar durante el backtracking  

    Un ejemplo de este algoritmo podria ser, tenes una caja de lápices de colores y quieres asegurarte de que puedes dibujar un arcoíris completo. Pero hay una regla: cada color solo puede ir al lado de ciertos colores. **El algoritmo AC-3** es como un amigo que revisa todos los lápices y se asegura de que cada uno tenga un vecino adecuado antes de empezar a dibujar. Así, cuando comiences a colorear, no te detendrás a mitad de camino porque todos los lápices están en el orden correcto para hacer un arcoíris perfecto.

    **referencias:**
    - [Algoritmo AC-3](https://en.wikipedia.org/wiki/AC-3_algorithm)
    - [Backtracking Algorithms](https://www.freecodecamp.org/news/backtracking-algorithms-recursive-search/)

    
2. Defina en sus propias palabras el término “Arc Consistency”  
    **En mis propias palabras, “Arc Consistency”** se refiere a un estado en el que, para cada par de variables en un CSP que comparten una restricción, cada valor de la primera variable tiene al menos un valor correspondiente en la segunda variable que satisface la restricción entre ellas. Esto asegura que no hay valores aislados que hagan imposible encontrar una solución completa al problema.

    Un ejemplo podria un juego de parejas de cartas. Cada carta tiene un número y debes encontrarle una pareja que tenga el mismo número. **“Arc Consistency”** significa que todas las cartas tienen al menos una pareja posible. Si alguna carta no tuviera pareja, no podrías ganar el juego. Entonces, antes de jugar, revisas todas las cartas para asegurarte de que cada una tiene una pareja y así sabes que el juego se puede ganar.

    **referencias:**
    - [Arc Consistency](https://en.wikipedia.org/wiki/Arc_consistency)
    - [Arc Consistency Explained](https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/)

### Task 2 - CSP con Backtracking, Beam y Local Search

In [1]:
from typing import Dict, List, Optional, Tuple, Callable
from itertools import permutations
import heapq
import random
import time

def revise(assignment: Dict[str, str], variable1: str, variable2: str, constraint: Callable[[str, str], bool]) -> bool:
    """
    Remove values from the domain of variable1 that do not satisfy the constraint with variable2.

    Args:
        assignment (Dict[str, str]): The current assignment of variables.
        variable1 (str): The first variable.
        variable2 (str): The second variable.
        constraint (Callable[[str, str], bool]): The constraint function.

    Returns:
        bool: True if the domain of variable1 is revised, False otherwise.
    """
    revised = False
    domain1 = domains[variable1]
    domain2 = domains[variable2]
    for value1 in domain1[:]:
        has_support = False
        for value2 in domain2:
            if constraint(value1, value2):
                has_support = True
                break
        if not has_support:
            domains[variable1].remove(value1)
            revised = True
    return revised

def ac3(csp: 'CSP') -> bool:
    """
    Perform the AC-3 algorithm to enforce arc consistency on the CSP.

    Args:
        csp ('CSP'): The CSP instance.

    Returns:
        bool: True if the CSP is arc consistent, False otherwise.
    """
    queue = [(variable1, variable2) for variable1 in csp.variables for variable2 in csp.variables if variable1 != variable2]
    while queue:
        variable1, variable2 = queue.pop(0)
        if revise(csp.assignment, variable1, variable2, csp.constraints[(variable1, variable2)]):
            if len(domains[variable1]) == 0:
                return False
            for variable3 in csp.variables - {variable1, variable2}:
                queue.append((variable3, variable1))
    return True

class CSP:
    def __init__(self, variables: List[str], domains: Dict[str, List[str]], constraints: Dict[Tuple[str, str], Callable[[str, str], bool]], assignment: Optional[Dict[str, str]] = None):
        """
        Initialize a Constraint Satisfaction Problem (CSP) instance.

        Args:
            variables (List[str]): The list of variables.
            domains (Dict[str, List[str]]): The domain of each variable.
            constraints (Dict[Tuple[str, str], Callable[[str, str], bool]]): The constraints between variables.
            assignment (Optional[Dict[str, str]], optional): The initial assignment of variables. Defaults to None.
        """
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        self.assignment = assignment or {}

# Variables (cursos)
variables = ['Curso1', 'Curso2', 'Curso3', 'Curso4', 'Curso5', 'Curso6', 'Curso7']

# Dominios (días posibles)
domains = {var: ['Lunes', 'Martes', 'Miércoles'] for var in variables}

# Asignación de estudiantes a cursos
student_courses = {
    'Estudiante1': ['Curso1', 'Curso6' ],
    'Estudiante2': ['Curso4', 'Curso3'],
    'Estudiante3': ['Curso7', 'Curso5'],
    'Estudiante4': ['Curso2',]
}

# Restricciones
def different_day_constraint(value1: str, value2: str) -> bool:
    """
    Check if two values represent different days.

    Args:
        value1 (str): The first value.
        value2 (str): The second value.

    Returns:
        bool: True if the values represent different days, False otherwise.
    """
    return value1 != value2

def same_course_different_day_constraint(value1: str, value2: str) -> bool:
    """
    Check if two values representing the same course have different days for different students.

    Args:
        value1 (str): The first value.
        value2 (str): The second value.

    Returns:
        bool: True if the values represent different days for different students, False otherwise.
    """
    for student, courses in student_courses.items():
        if value1 in courses and value2 in courses:
            return value1 != value2
    return True

constraints = {}
for variable1, variable2 in permutations(variables, 2):
    constraints[(variable1, variable2)] = different_day_constraint
    for student, courses in student_courses.items():
        if variable1 in courses and variable2 in courses:
            constraints[(variable1, variable2)] = same_course_different_day_constraint
            break
        
# Crear una instancia del problema CSP
exam_scheduling_csp = CSP(variables, domains, constraints)



- Todos los examenes, tienen que ser en dias diferentes.
- Los estudiantes solo pueden tener UN examen por dia.
- Los estudiantes miembros de un mismo curso no tengan exámenes el mismo día.



#### Backtracking implementation

In [2]:

def backtracking_search(csp: 'CSP', assignment: Optional[Dict[str, str]] = None) -> Optional[Dict[str, str]]:
    """
    Perform backtracking search to find a solution for the given CSP.

    Args:
        csp (CSP): The Constraint Satisfaction Problem to solve.
        assignment (Optional[Dict[str, str]]): The current assignment of variables (default: None).

    Returns:
        Optional[Dict[str, str]]: A valid assignment that satisfies all constraints, or None if no solution is found.
    """
    # If no assignment is provided, initialize an empty assignment
    if assignment is None:
        assignment = {}

    # If the assignment is complete (all variables are assigned), return the assignment
    if len(assignment) == len(csp.variables):
        return assignment

    # Get the list of unassigned variables
    unassigned_variables = [v for v in csp.variables if v not in assignment]

    # Iterate through the unassigned variables
    for variable in unassigned_variables:
        # Iterate through the domain values of the current variable
        for value in domains[variable]:
            # Create a new assignment by copying the current assignment and assigning the value to the variable
            new_assignment = assignment.copy()
            new_assignment[variable] = value

            # Check if the new assignment satisfies the constraints using the AC3 algorithm
            if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
                # Recursively call the backtracking search with the new assignment
                result = backtracking_search(CSP(csp.variables, csp.domains, csp.constraints, new_assignment), new_assignment)
                # If a valid assignment is found, return the result
                if result is not None:
                    return result

        # If no valid assignment is found for the current variable, backtrack to the previous variable
        break

    # If no valid assignment is found for any variable, return None
    return None


### Beam search implementation

In [3]:
def beam_search(csp, beam_width, assignment={}):
    """
    Performs beam search to find a solution for the given constraint satisfaction problem (CSP).

    Args:
        csp (CSP): The constraint satisfaction problem to solve.
        beam_width (int): The width of the beam.
        assignment (dict, optional): The current assignment of variables (default is an empty dictionary).

    Returns:
        dict or None: A valid assignment that satisfies all constraints, or None if no solution is found.
    """

    # Check if the assignment is complete
    if len(assignment) == len(csp.variables):
        return assignment

    # Get the list of unassigned variables
    unassigned_variables = [v for v in csp.variables if v not in assignment]

    # Create an empty beam
    beam = []

    # Iterate over each unassigned variable
    for variable in unassigned_variables:
        # Iterate over each value in the domain of the variable
        for value in csp.domains[variable]:
            # Create a new assignment by copying the current assignment
            new_assignment = assignment.copy()
            # Assign the value to the variable in the new assignment
            new_assignment[variable] = value

            # Check if the new assignment satisfies the constraints using the AC3 algorithm
            if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
                # Calculate the number of conflicts in the new assignment
                conflicts = sum(1 for (var1, var2), constraint in csp.constraints.items()
                                if var1 in new_assignment and var2 in new_assignment
                                and not constraint(new_assignment[var1], new_assignment[var2]))
                # Add the new assignment to the beam with its conflicts as the priority
                heapq.heappush(beam, (conflicts, tuple(new_assignment.items())))

    # Select the top 'beam_width' assignments from the beam
    beam = heapq.nsmallest(beam_width, beam)

    # Recursively search for a solution using each assignment in the beam
    for _, new_assignment_tuple in beam:
        new_assignment = dict(new_assignment_tuple)
        result = beam_search(CSP(csp.variables, csp.domains, csp.constraints, new_assignment), beam_width, new_assignment)
        if result is not None:
            return result

    # If no solution is found, return None
    return None


### Local search implementation

In [4]:
def local_search(csp, max_iterations, assignment={}):
    """
    Performs local search on a Constraint Satisfaction Problem (CSP).

    Args:
        csp (CSP): The Constraint Satisfaction Problem to solve.
        max_iterations (int): The maximum number of iterations to perform.
        assignment (dict, optional): The initial assignment of variables. Defaults to an empty dictionary.

    Returns:
        dict or None: The assignment that satisfies the CSP constraints, or None if no solution is found.
    """
    # If no initial assignment is provided, randomly assign values to variables
    if len(assignment) == 0:
        assignment = {variable: random.choice(csp.domains[variable]) for variable in csp.variables}

    # Perform local search for a maximum number of iterations
    for _ in range(max_iterations):
        # Apply the AC-3 algorithm to enforce arc consistency
        if ac3(CSP(csp.variables, csp.domains, csp.constraints, assignment)):
            return assignment

        # Randomly select a variable from the assignment
        variable = random.choice(list(assignment.keys()))

        # Randomly select a value for the selected variable that is different from its current assignment
        value = random.choice([v for v in csp.domains[variable] if v != assignment[variable]])

        # Create a new assignment by copying the current assignment and updating the selected variable's value
        new_assignment = assignment.copy()
        new_assignment[variable] = value

        # Apply the AC-3 algorithm to enforce arc consistency with the new assignment
        if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
            assignment = new_assignment

    # Return None if no solution is found within the maximum number of iterations
    return None


### Use case and comparison of solution using algorithms

In [5]:
from prettytable import PrettyTable
import time

def run_exam_scheduling_algorithms(exam_scheduling_csp):
    """
    Runs different algorithms to solve the exam scheduling problem and collects the results.

    Args:
        exam_scheduling_csp (CSP): The exam scheduling constraint satisfaction problem.

    Returns:
        list: A list of tuples containing the algorithm name, solution, execution time, and validity flag.
    """
    algorithms = ['Backtracking Search', 'Beam Search', 'Local Search']
    results = []

    for algorithm in algorithms:
        start_time = time.time()

        if algorithm == 'Backtracking Search':
            solution = backtracking_search(exam_scheduling_csp)
        elif algorithm == 'Beam Search':
            beam_width = 5
            solution = beam_search(exam_scheduling_csp, beam_width)
        else:  # Local Search
            max_iterations = 1000
            solution = local_search(exam_scheduling_csp, max_iterations)

        end_time = time.time()
        execution_time = end_time - start_time

        if solution is not None:
            is_valid = ac3(CSP(exam_scheduling_csp.variables, exam_scheduling_csp.domains, exam_scheduling_csp.constraints, solution))
        else:
            is_valid = False

        results.append((algorithm, solution, execution_time, is_valid))

    return results

def display_results(results):
    """
    Displays the results of the exam scheduling algorithms in a table using prettytable.

    Args:
        results (list): A list of tuples containing the algorithm name, solution, execution time, and validity flag.
    """
    table = PrettyTable()
    table.field_names = ['Algoritmo', 'Solución', 'Tiempo de ejecución (segundos)', '¿Solución válida?']
    table.align = 'l'  # Align everything to the left
    table.float_format = '.10'  # Set float format to 10 digits

    for algorithm, solution, execution_time, is_valid in results:
        if solution is None:
            solution_str = 'No se encontró solución'
        else:
            solution_str = '\n'.join(f"{course}: {day}" for course, day in solution.items())

        table.add_row([algorithm, solution_str, execution_time, is_valid])

    print(table)

# Usage:
results = run_exam_scheduling_algorithms(exam_scheduling_csp)
display_results(results)


+---------------------+-------------------+--------------------------------+-------------------+
| Algoritmo           | Solución          | Tiempo de ejecución (segundos) | ¿Solución válida? |
+---------------------+-------------------+--------------------------------+-------------------+
| Backtracking Search | Curso1: Lunes     | 0.0005125999                   | True              |
|                     | Curso2: Lunes     |                                |                   |
|                     | Curso3: Lunes     |                                |                   |
|                     | Curso4: Lunes     |                                |                   |
|                     | Curso5: Lunes     |                                |                   |
|                     | Curso6: Lunes     |                                |                   |
|                     | Curso7: Lunes     |                                |                   |
| Beam Search         | Curso1